In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from crewai import Agent, Crew, Process, Task, LLM
from crewai_tools import MCPServerAdapter
from crewai_tools import FileWriterTool
from crewai_tools import FileReadTool
from mcp import StdioServerParameters
import yaml
import os
from pathlib import Path
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, FileReadTool,FileWriterTool,DirectoryReadTool

In [ ]:
# Load environment variables
env_path = "backend//.env"
load_dotenv(dotenv_path=env_path)

In [ ]:
# Skip MCP server setup due to Jupyter compatibility issues
# The MCP server has known issues with Jupyter due to fileno() not being supported
# Instead, we'll use the custom Playwright tools directly from the backend

print("⚠️  Skipping MCP server setup due to Jupyter notebook compatibility issues")
print("✅ Using custom Playwright tools instead")

# Load custom tools from backend
import sys
sys.path.append(os.path.abspath('backend'))

try:
    from backend.tools.mcp_playwright_tool import (
        MyCustomTool, 
        PlaywrightNavigateTool, 
        PlaywrightScreenshotTool, 
        PlaywrightClickTool, 
        PlaywrightFillTool, 
        PlaywrightGetTextTool
    )
    
    # Setup tools directly without MCP server
    from crewai_tools import FileWriterTool, FileReadTool, DirectoryReadTool
    
    base_tools = [FileWriterTool(), FileReadTool(), DirectoryReadTool()]
    playwright_tools = [
        MyCustomTool(),
        PlaywrightNavigateTool(),
        PlaywrightScreenshotTool(), 
        PlaywrightClickTool(),
        PlaywrightFillTool(),
        PlaywrightGetTextTool()
    ]
    
    # Combine all tools
    tools = base_tools + playwright_tools
    
    print(f"✅ Successfully loaded {len(tools)} tools:")
    print(f"   - {len(base_tools)} base CrewAI tools")
    print(f"   - {len(playwright_tools)} custom Playwright tools")
    print("\n🔧 Available tools:")
    for i, tool in enumerate(tools, 1):
        tool_name = tool.__class__.__name__
        print(f"   {i}. {tool_name}")
    
    print(f"\n✅ Tools setup complete! Ready for CrewAI agents.")
    
except ImportError as e:
    print(f"❌ Error importing custom tools: {e}")
    print("⚠️  Falling back to basic CrewAI tools only")
    tools = [FileWriterTool(), FileReadTool(), DirectoryReadTool()]
    print(f"✅ Loaded {len(tools)} basic tools")

except Exception as e:
    print(f"❌ Unexpected error: {e}")
    tools = []

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'backend//config//agents.yaml',
    'tasks': 'backend//config//tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

print(f"{len(agents_config)} agents loaded")
print(f"{len(tasks_config)} tasks loaded")


In [ ]:
os.environ['AZURE_DEPLOYMENT_TARGET_URL'] = os.getenv("AZURE_API_BASE")
os.environ['AZURE_API_KEY'] = os.getenv("AZURE_API_KEY")
os.environ['AZURE_API_VERSION'] = os.getenv("AZURE_API_VERSION")
azure_llm = "azure/gpt-4.1"

In [ ]:
from backend.tools.mcp_playwright_tool import MyCustomTool  # Import the custom Playwright tool

project_manager_agent = Agent(
  config=agents_config['pm'],
  llm=azure_llm,
 )

discovery_agent = Agent(
  config=agents_config['discovery'],
  llm=azure_llm,
  tools=[FileReadTool(), FileWriterTool(), DirectoryReadTool(), MyCustomTool()] # Ensure Playwright/MCP tool is available
)

In [ ]:
discover_application = Task(
  description=tasks_config['discover_application']['description'],
  expected_output=tasks_config['discover_application']['expected_output'],
  agent=discovery_agent,
  output_file=tasks_config['discover_application'].get('output_file')
)

build_site_map = Task(
  description=tasks_config['build_site_map']['description'],
  expected_output=tasks_config['build_site_map']['expected_output'],
  agent=discovery_agent,
  context=[discover_application],
  output_file=tasks_config['build_site_map'].get('output_file')
)

validate_discovery = Task(
  description=tasks_config['validate_discovery']['description'],
  expected_output=tasks_config['validate_discovery']['expected_output'],
  context=[build_site_map],
  agent=project_manager_agent
)

In [ ]:
# Creating Crew
crew = Crew(
  agents=[project_manager_agent, discovery_agent],
  tasks=[ discover_application, build_site_map, validate_discovery ],
  llm=azure_llm,
  verbose=True
)

In [ ]:
# Load mandatory instructions
instructions_file_path = 'backend//agent_instructions//mandatory_pre_steps.txt'
with open(instructions_file_path, 'r') as file:
    instructions = file.read()

# The given Python dictionary
inputs = {
  'BASE_URL': 'www.google.com',
  'HEADLESS': "false",
  'SAMPLE_DIR': 'debug_samples',
  'INSTRUCTIONS': instructions
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

In [ ]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics